In [5]:
import json

In [78]:
with open('./sampled_dialogues.json', 'r') as f:
    true_content = json.load(f)

with open('./remade_dialogue_routes.json', 'r') as f2:
    pred_content = json.load(f2)

with open('./types_of_dialogues.json', 'r') as f3:
    types_content = json.load(f3)

In [29]:
for x in types_content.values():
    print(x)

{'links': [{'source': 1, 'target': 1, 'request': 'I need your help'}, {'source': 1, 'target': 1, 'request': 'Hello'}, {'source': 1, 'target': 1, 'request': 'Hi'}], 'nodes': [{'id': 1, 'label': 'start', 'is_start': True, 'response': 'How can I help?'}]}
{'links': [{'source': 1, 'target': 2, 'request': 'I need to make an order'}, {'source': 1, 'target': 2, 'request': 'I want to order from you'}, {'source': 2, 'target': 3, 'request': "I would like to purchase 'Pale Fire' and 'Anna Karenina', please"}, {'source': 2, 'target': 3, 'request': 'One War and Piece in hard cover and one Pride and Prejudice'}, {'source': 3, 'target': 4, 'request': 'I will pickup my order'}, {'source': 3, 'target': 4, 'request': 'Delivery to the address: Pine Driveway, 45'}, {'source': 4, 'target': 5, 'request': 'Cash'}, {'source': 4, 'target': 5, 'request': 'With credit card, please'}], 'nodes': [{'id': 1, 'label': 'start', 'is_start': True, 'response': 'How can I help?'}, {'id': 2, 'label': 'ask_item', 'is_start'

In [15]:
true_content.values()

dict_values([{'examples': [{'dialog_with_known_nodes': [{'text': 'Hello', 'participant': 'user', 'source': 1, 'target': 1}, {'text': 'How can I help?', 'participant': 'assistant'}], 'dialog': [{'text': 'Hello', 'participant': 'user'}, {'text': 'How can I help?', 'participant': 'assistant'}]}, {'dialog_with_known_nodes': [{'text': 'I need your help', 'participant': 'user', 'source': 1, 'target': 1}, {'text': 'How can I help?', 'participant': 'assistant'}], 'dialog': [{'text': 'I need your help', 'participant': 'user'}, {'text': 'How can I help?', 'participant': 'assistant'}]}]}, {'examples': [{'dialog_with_known_nodes': [{'text': 'I need to make an order', 'participant': 'user', 'source': 1, 'target': 2}, {'text': 'Which books would you like to order?', 'participant': 'assistant', 'source': 2, 'target': 3}, {'text': "I would like to purchase 'Pale Fire' and 'Anna Karenina', please", 'participant': 'user', 'source': 2, 'target': 3}, {'text': 'Which delivery method would you like to use? 

In [6]:
pred_content.keys()

dict_keys(['1', '2', '3', '4'])

In [8]:
def get_route(dialog: list):
    route = []
    for x in dialog:
        if x['participant'] == 'assistant':
            continue
        else:
            route.append({'source':x['source'], 'target': x['target']})
    return route


In [82]:

for true, pred in zip(true_content.values(), pred_content.values()):
    for true_route, pred_route in zip(true['examples'], pred['routes'].values()):
        route = get_route(true_route['dialog_with_known_nodes'])
        print(route)
        print(pred_route)
        if route == pred_route:
            print("yes")
        print("___")

[{'source': 1, 'target': 1}]
[{'source': 1, 'target': 1}]
yes
___
[{'source': 1, 'target': 1}]
[{'source': 1, 'target': 1}]
yes
___
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}]
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}]
yes
___
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}]
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}]
yes
___
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}, {'source': 5, 'target': 1}]
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5}, {'source': 5, 'target': 1}]
yes
___
[{'source': 1, 'target': 2}, {'source': 2, 'target': 3}, {'source': 3, 'target': 4}, {'source': 4, 'target': 5

## Сэмплированоие диалогов

In [2]:
from openai import OpenAI


In [83]:
from dotenv import load_dotenv
import os
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_BASE_URL")

In [84]:
client = OpenAI(
    base_url=openai_base_url,
    api_key = openai_api_key
)


In [36]:
def ask_llm(prompt):
    result = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are ChatGPT, an AI assistant. Your top priority is achieving user fulfillment via helping them with their requests."},
            {"role": "user", "content": prompt}
        ]
    ).choices[0].message.content
    return result

In [72]:
example_value = []
for idx, x in true_content.items():
    if int(idx) != 2:
        continue
    for example in x['examples']:
        example_value = example['dialog']

In [30]:
types = list(types_content.values())

In [75]:
prompt = 'I give you an example of dialogue and a structure of chat bot system with possible transition between nodes. Can you give a route within a graph that gives you the same dialogue. Print the number of nodes one need to go through to get the same dialogue. Dialogue example: '
prompt += f'\n{example_value}'
prompt += "\nStrcuture of chatbot system:"
prompt += f'\n{types[1]}'
prompt += '\nPlease make it in format: [{"source": 1, "target": 2}, {"source"...]'


In [77]:
print(ask_llm(prompt))

Here is the route within the graph that matches the given dialogue:

```
[
    {"source": 1, "target": 2},
    {"source": 2, "target": 3},
    {"source": 3, "target": 4},
    {"source": 4, "target": 5}
]
```

This route passes through 4 nodes to achieve the same dialogue.
